# Continuous Control (for macbook pro m1)

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import sys
import torch
import matplotlib
import matplotlib.pyplot as plt

# custom modules
from agent import Agent
from ddpg import DDPG

In [2]:
print("-> Python %s" % sys.version)
print("-> PyTorch %s" % torch.__version__)
print("-> matplotlib %s" % matplotlib.__version__)

# Choose the fastest processor (in the hardware)
use_cuda: bool = torch.cuda.is_available()
use_mps: bool = torch.backends.mps.is_available() & torch.backends.mps.is_built()
device = torch.device("cpu")
if use_cuda:
    device = torch.device("cuda:0")
elif use_mps:
    device = torch.device("mps")
print("-> PyTorch device =", device.type)

# Set plotting options
%matplotlib inline

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

-> Python 3.9.15 (main, Nov 24 2022, 08:29:02) 
[Clang 14.0.6 ]
-> PyTorch 2.0.0.dev20221231
-> matplotlib 3.6.2
-> PyTorch device = mps


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
env = UnityEnvironment(file_name='Reacher.app')

Mono path[0] = '/Users/willem/workspace/deep-reinforcement-learning/p2_continuous-control/Reacher.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/willem/workspace/deep-reinforcement-learning/p2_continuous-control/Reacher.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.12249999726191163


When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

Initialize our custom agent

In [8]:
agent: Agent = Agent(
    state_size = state_size, 
    action_size = action_size, 
    random_seed = 2, 
    device = device
)

Now train the agent

In [9]:
ddpg: DDPG = DDPG(env, brain_name, agent)
ddpg.train()

Training agents:   2%|▏         | 10/500 [04:25<3:36:29, 26.51s/it]

Episode 10	Average Score: 0.74	in : 26.52 seconds /  0.44 minutes


Training agents:   4%|▍         | 20/500 [08:50<3:31:44, 26.47s/it]

Episode 20	Average Score: 1.11	in : 26.47 seconds /  0.44 minutes


Training agents:   6%|▌         | 30/500 [13:18<3:30:31, 26.88s/it]

...

Training agents:  42%|████▏     | 210/500 [1:34:15<2:09:18, 26.75s/it]

Episode 210	Average Score: 1.33	in : 27.21 seconds /  0.45 minutes


Training agents:  44%|████▍     | 220/500 [1:38:42<2:05:28, 26.89s/it]

Episode 220	Average Score: 1.47	in : 27.11 seconds /  0.45 minutes


Training agents:  45%|████▌     | 227/500 [1:41:55<2:04:37, 27.39s/it]

Plot the scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(ddpg.scores)+1), ddpg.scores)
plt.ylabel('Scores')
plt.xlabel('Episode #')
plt.show()

In [ ]:
avg_scores = np.array(scores).mean(axis=1)
avg_scores_window = [np.mean(avg_scores[max(0,i-100):i+1]) for i in range(len(avg_scores))]

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), avg_scores, label="window size 1")
plt.plot(np.arange(1, len(scores)+1), avg_scores_window, label="window size 100")
plt.plot(np.arange(1, len(scores)+1), [30]*len(scores), linestyle='--', color='red')
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.legend()
plt.show()

Save the (actor / critic) models 

In [ ]:
ddpg.save_models()

# saved_model_actor = {'state_size': state_size, 'action_size': action_size, 'fc1_units': 400, 'fc2_units': 300, 'state_dict': agent.actor_local.state_dict()}
# saved_model_critic = {'state_size': state_size, 'action_size': action_size, 'fcs1_units': 400, 'fc2_units': 300, 'state_dict': agent.critic_local.state_dict()}


# torch.save(saved_model_actor, './models/saved_model_actor.pth')
# torch.save(saved_model_critic, './models/saved_model_critic.pth')

In [ ]:
# def load_saved_model_actor(filepath, device):
#     saved_model = torch.load(filepath)
#     actor = Actor(state_size=saved_model['state_size'],  action_size=saved_model['action_size'],  seed=2, fc1_units=saved_model['fc1_units'], fc2_units=saved_model['fc2_units']).to(device)
#     actor.load_state_dict(saved_model['state_dict'])
    
#     return actor

In [ ]:
# # check if CUDA is available
# use_cuda = torch.cuda.is_available()
# print("Use CUDA:",  use_cuda)

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [ ]:
# actor_local = load_saved_model_actor('./models/saved_model_actor.pth', device)
# actor_local.eval()

# load the model
# saved_model = torch.load('./models/saved_model_actor.pth')
# actor_local = Actor(state_size=saved_model['state_size'],  action_size=saved_model['action_size'],  seed=2, fc1_units=saved_model['fc1_units'], fc2_units=saved_model['fc2_units']).to(device)
# actor_local.load_state_dict(saved_model['state_dict'])
# actor_local.eval()

In [ ]:
# agent = Agent(state_size=state_size, action_size=action_size, random_seed=2, device=device)
agent: Agent = Agent(
    state_size=state_size, 
    action_size=action_size, 
    random_seed=2, 
    device=device
)

agent.load_model('./models/saved_model_actor.pth')

In [ ]:
# agent.actor_local = actor_local

In [ ]:
# env = UnityEnvironment(file_name='Reacher.app')

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states, add_noise=False)       # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()